In [1]:
import datetime
import os
import sys
import time
import collections
import argparse
import torch
import torch.utils.data
from torch import nn
import h5py
from tqdm import tqdm

import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib
from pytorch_quantization.tensor_quant import QuantDescriptor
from utils import customDataset
from EDSR_Quant.model import EDSR
from RRDBNet_Quant.model import RRDBNet
from sklearn.model_selection import KFold


E0915 00:17:57.726973 140157013657408 amp_wrapper.py:31] AMP is not avaialble.


In [2]:
quant_desc_input = QuantDescriptor(calib_method='histogram', axis=None)
quant_nn.QuantConv2d.set_default_quant_desc_input(quant_desc_input)
quant_desc_weight = QuantDescriptor(calib_method='histogram', axis=None)
quant_nn.QuantConv2d.set_default_quant_desc_weight(quant_desc_weight)

In [3]:
parser = argparse.ArgumentParser(description="SuperRes Pytorch")
parser.add_argument("--model", type=str, help="model name")
parser.add_argument("--ckpt", type=str, help="checkpoint path")
parser.add_argument("--kfoldIndex", default=0, type=int, help="start kfold valid index 0 ~ 9")
opt = parser.parse_args()
if opt.model is None:
        print("model name?")
elif not torch.cuda.is_available():
    print("cuda is not available")
else:
    print("===> Loading model")
    if opt.model == "EDSR":
        model = EDSR(opt.scale, lr=opt.lr, batch_size=opt.batchSize)
        print("Model EDSR is loaded")
    elif opt.model == "RRDBNet":
        model = RRDBNet(64, 23, scale_factor=opt.scale, lr=opt.lr, batch_size=opt.batchSize)
        print("Model RRDBNet is loaded")

model = model.load_from_checkpoint(opt.ckpt)
model.cuda()
model.eval()

RRDBNet(
  (input_conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (rrdb): Sequential(
    (0): ResidualInResidualDenseBlock(
      (layer1): ResidualDenseBlock(
        (layer1): Sequential(
          (0): QuantConv2d(
            64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
            (_input_quantizer): TensorQuantizer(8bit narrow fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
            (_weight_quantizer): TensorQuantizer(8bit narrow fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
          )
          (1): LeakyReLU(negative_slope=0.2, inplace=True)
        )
        (layer2): Sequential(
          (0): QuantConv2d(
            96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
            (_input_quantizer): TensorQuantizer(8bit narrow fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
            (_weight_quantizer): TensorQuantizer(8bit narrow fak

In [4]:
print("===> Loading datasets")

# change dir here
hf = h5py.File("/media/jacob/Elements/thermal-images-datasets/FLIR_ADAS_1_3/merged/output/FLIR16_x"+str(4)+".hdf5", 'r')
inputs = hf.get("input")
labels = hf.get("label")

kf = KFold(n_splits=10)
kf_index = 0

for trainFold_indexes, testFold_indexes in kf.split(inputs):
    if kf_index < opt.kfoldIndex:
        kf_index+=1
        continue
    print("===> Start K-FOLD Index " + str(kf_index))

    train_dataset = customDataset(inputs[trainFold_indexes], labels[trainFold_indexes])
    test_dataset = customDataset(inputs[testFold_indexes], labels[testFold_indexes])

    training_dataloader = DataLoader(dataset=train_dataset, num_workers=8, batch_size=16, shuffle=True)
    testing_dataloader = DataLoader(dataset=test_dataset, num_workers=8, batch_size=16)

    break

===> Loading datasets
===> Start K-FOLD Index 0


In [5]:


def collect_stats(model, data_loader, num_batches):
    """Feed data to the network and collect statistic"""

    # Enable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.disable_quant()
                module.enable_calib()
            else:
                module.disable()

    for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
        model(image.cuda())
        if i >= num_batches:
            break

    # Disable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.enable_quant()
                module.disable_calib()
            else:
                module.enable()
            
def compute_amax(model, **kwargs):
    # Load calib result
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                if isinstance(module._calibrator, calib.MaxCalibrator):
                    module.load_calib_amax()
                else:
                    module.load_calib_amax(**kwargs)
            #print(F"{name:40}: {module}")
    model.cuda()



In [6]:
#test multiple calib methods and pick the best one to save
with torch.no_grad():
    collect_stats(model, training_dataloader, num_batches=64)
    compute_amax(model, method="percentile", percentile=99.9999)

100%|██████████| 64/64 [3:10:45<00:00, 178.83s/it]
W0915 03:28:54.717801 140157013657408 tensor_quantizer.py:173] Disable HistogramCalibrator
W0915 03:28:54.718474 140157013657408 tensor_quantizer.py:173] Disable HistogramCalibrator
W0915 03:28:54.718885 140157013657408 tensor_quantizer.py:173] Disable HistogramCalibrator
W0915 03:28:54.719257 140157013657408 tensor_quantizer.py:173] Disable HistogramCalibrator
W0915 03:28:54.719624 140157013657408 tensor_quantizer.py:173] Disable HistogramCalibrator
W0915 03:28:54.719974 140157013657408 tensor_quantizer.py:173] Disable HistogramCalibrator
W0915 03:28:54.720334 140157013657408 tensor_quantizer.py:173] Disable HistogramCalibrator
W0915 03:28:54.721221 140157013657408 tensor_quantizer.py:173] Disable HistogramCalibrator
W0915 03:28:54.721603 140157013657408 tensor_quantizer.py:173] Disable HistogramCalibrator
W0915 03:28:54.721968 140157013657408 tensor_quantizer.py:173] Disable HistogramCalibrator
W0915 03:28:54.722348 140157013657408 t

In [7]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1)
trainer.validate(model, dataloaders=testing_dataloader)

I0915 03:28:59.732511 140157013657408 distributed.py:90] GPU available: True, used: True
I0915 03:28:59.733161 140157013657408 distributed.py:90] TPU available: False, using: 0 TPU cores
I0915 03:28:59.733516 140157013657408 distributed.py:90] IPU available: False, using: 0 IPUs
I0915 03:28:59.734261 140157013657408 gpu.py:54] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jacob/.local/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:373: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  f"Your {mode}_dataloader has `shuffle=True`, it is best practice to turn"


Validating: 100%|██████████| 256/256 [02:15<00:00,  1.90it/s]--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'psnr': 46.499271392822266,
 'val_L1loss': 66.49312591552734,
 'val_L2loss': 7671.87890625}
--------------------------------------------------------------------------------


[{'psnr': 46.499271392822266,
  'val_L1loss': 66.49312591552734,
  'val_L2loss': 7671.87890625}]

In [12]:
with torch.no_grad():
    compute_amax(model, method="percentile", percentile=99.99)
    trainer.validate(model, dataloaders=testing_dataloader)

W0915 15:30:49.167002 140157013657408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0915 15:30:49.167884 140157013657408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0915 15:30:49.168576 140157013657408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0915 15:30:49.169400 140157013657408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0915 15:30:49.170192 140157013657408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0915 15:30:49.170967 140157013657408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0915 15:30:49.171812 140157013657408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0915 15:30:49.172519 140157013657408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0915 15:30:49.173265 140157013657408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0915 15:30:49.173903 140157013657408

Validating: 100%|██████████| 256/256 [02:04<00:00,  2.17it/s]--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'psnr': 50.1746826171875,
 'val_L1loss': 40.08448791503906,
 'val_L2loss': 3097.244873046875}
--------------------------------------------------------------------------------


In [14]:
trainer.save_checkpoint(opt.model+"_Quant.ckpt")